In [3]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly  --apt-packages libomp5 libopenblas-dev

curl: /opt/conda/lib/libcurl.so.4: no version information available (required by curl)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6034  100  6034    0     0  33513      0 --:--:-- --:--:-- --:--:-- 33709
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-nightly ...
Found existing installation: torch 1.7.1+cpu
Uninstalling torch-1.7.1+cpu:
  Successfully uninstalled torch-1.7.1+cpu
Found existing installation: torchvision 0.8.2+cpu
Uninstalling torchvision-0.8.2+cpu:
  Successfully uninstalled torchvision-0.8.2+cpu
Copying gs://tpu-pytorch/wheels/colab/torch-nightly-cp37-cp37m-linux_x86_64.whl...
| [1 files][112.5 MiB/112.5 MiB]                                                
Operation completed over 1 objects/112.5 MiB.                                    
Copying gs://tpu-pytorch/wheels/colab/torch_xla-nightly-cp37-cp37m-linux_x86_64.whl...
-

In [4]:
import os

os.environ['XLA_USE_BF16'] = "1"
os.environ['XLA_TENSOR_ALLOCATOR_MAXSIZE'] = '100000000'

import torch
import pandas as pd
from scipy import stats
import numpy as np

import gc

from tqdm import tqdm
from collections import OrderedDict, namedtuple
import torch.nn as nn
from torch.optim import lr_scheduler
import joblib
from joblib import Parallel, delayed

import torch_xla.utils.serialization as xser

import time

import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.autograd import Variable
from torch.utils.data.sampler import SequentialSampler, RandomSampler
import sklearn

import logging
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup, get_constant_schedule, XLMRobertaTokenizer, XLMRobertaModel, XLMRobertaConfig, get_cosine_schedule_with_warmup
import sys
from sklearn import metrics, model_selection
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm

from random import shuffle
import random

import re

import warnings
import torch_xla
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.utils.utils as xu
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.test.test_utils as test_utils
import warnings
warnings.filterwarnings("ignore")

2022-10-09 12:00:40.400847: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib


In [5]:
data = pd.read_csv("../input/datalang/data1.csv")

In [6]:
data

,Unnamed: 0,text,lang
0,0,екі мың бес-екі мың алтыншы жылдары қазақстан ...,0
1,1,туристерге қолайлы жағдай жасау біздің басты м...,0
2,2,алдағы уақытта ауыл тұрғындарының тұрмыстық жа...,0
3,3,бақылауында ұстайды.,0
4,4,"шаршамаңдар, көп рақмет!",0
...,...,...,...
256900,256900,А благодарность? Для чего? Это тебя мы должны ...,1
256901,256901,Ты уходишь от нас?,1
256902,256902,О нет. Мы всегда будем вместе.,1
256903,256903,"Совет кардиналов! Я так нервничаю! Что, если я...",1


In [7]:
gc.collect()
!free -h

              total        used        free      shared  buff/cache   available
Mem:            17G        892M         10G        1.0M        6.5G         16G
Swap:            0B          0B          0B


In [8]:
MAX_LEN = 224

In [9]:
def onehot(size, target):
    vec = torch.zeros(size, dtype=torch.float32)
    vec[target] = 1.
    return vec

class DatasetRetriever(Dataset):

    def __init__(self, df):
        self.texts = df['text'].values
        self.labels = df['lang'].values
        self.tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

    def get_tokens(self, text):
        encoded = self.tokenizer.encode_plus(text, add_special_tokens=True,max_length=MAX_LEN,pad_to_max_length=True)
        return encoded['input_ids'], encoded['attention_mask']

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = torch.tensor(int(self.labels[idx])).long()
        target = onehot(2, label)
        tokens, attention_mask = self.get_tokens(text)
        tokens, attention_mask = torch.tensor(tokens), torch.tensor(attention_mask)

        return target, tokens, attention_mask

In [11]:
class CustomRoberta(nn.Module):
    def __init__(self):
        super(CustomRoberta, self).__init__()
        self.num_labels = 2
        self.roberta = transformers.XLMRobertaModel.from_pretrained("xlm-roberta-base", output_hidden_states=False, num_labels=1, return_dict=False)
        self.dropout = nn.Dropout(p=0.3)
        self.linear = nn.Linear(
            in_features=self.roberta.pooler.dense.out_features*2,
            out_features=2,
        )
        

    def forward(self,
                input_ids=None,
                attention_mask=None,
                position_ids=None,
                head_mask=None,
                inputs_embeds=None):

        o1, o2 = self.roberta(input_ids,
                               attention_mask=attention_mask,
                               position_ids=position_ids,
                               head_mask=head_mask,
                               inputs_embeds=inputs_embeds)
        apool = torch.mean(o1, 1)
        mpool, _ = torch.max(o1, 1)
        x = torch.cat((apool, mpool), 1)
        x = self.dropout(x)
        return self.linear(x)

In [12]:
mx = CustomRoberta()

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [13]:
os.makedirs("./inference")

In [14]:
mx.roberta.save_pretrained("./inference")

In [15]:
def loss_fn(x, target):
    x = x.float()
    target = target.float()
    logprobs = torch.nn.functional.log_softmax(x, dim = -1)
    nll_loss = -logprobs * target
    nll_loss = nll_loss.sum(-1)
    smooth_loss = -logprobs.mean(dim=-1)
    loss = 0.9 * nll_loss + 0.1 * smooth_loss
    return loss.mean()


def reduce_fn(vals):
    return sum(vals) / len(vals)


In [16]:
def train_loop_fn(trainloader, model, optimizer, device, scheduler=None):
    model.train()
    model = model.to(device)
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    counter = 0
    for step, (targets, inputs, attention_masks) in enumerate(trainloader):
        inputs = inputs.to(device, dtype=torch.long)
        attention_masks = attention_masks.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)
        optimizer.zero_grad()
        output = model(input_ids=inputs, attention_mask=attention_masks)
        loss = loss_fn(output, targets)
        correct_predictions += (output.argmax(1) == targets.argmax(1)).type(torch.float).sum().item()
        total_predictions += len(targets)
        if step % 50 == 0:
            loss_reduced = xm.mesh_reduce('loss_reduce', loss, reduce_fn)
            xm.master_print(f"step: {step} {correct_predictions / total_predictions}, {loss_reduced}")
        loss.backward()
        xm.optimizer_step(optimizer)
        if scheduler is not None:
            scheduler.step()
    xm.master_print(", accuracy = [{}]".format(correct_predictions / total_predictions))
    

In [17]:
def eval_loop_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    counter = 0
    for step, (targets, inputs, attention_masks) in enumerate(data_loader):
        with torch.no_grad():
            inputs = inputs.to(device, dtype=torch.long)
            attention_masks = attention_masks.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)
            outputs = model(input_ids=inputs, attention_mask=attention_masks)
            correct_predictions += (outputs.argmax(1) == targets.argmax(1)).type(torch.float).sum().item()
            total_predictions += len(targets)
            targets_np = targets.cpu().detach().tolist()
            outputs_np = outputs.cpu().detach().tolist()
            fin_targets.extend(targets_np)
            fin_outputs.extend(outputs_np)    
            del targets_np, outputs_np
            gc.collect()
    return fin_outputs, fin_targets

In [18]:
data1 = pd.read_csv("../input/datalang/test_data.csv")
data1['lang'] = np.where(data1['lang'] == 'kz', 0, 1)

In [19]:
train_dataset = DatasetRetriever(data)
valid_dataset = DatasetRetriever(data1)

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [20]:
def _run():
    MAX_LEN = 192
    BATCH_SIZE = 16 
    EPOCHS = 5 
    train_sampler = torch.utils.data.distributed.DistributedSampler(
          train_dataset,
          num_replicas=xm.xrt_world_size(),
          rank=xm.get_ordinal(), 
          shuffle=True)

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        sampler=train_sampler,
        drop_last=True,
        num_workers=0,
    )
    valid_sampler = torch.utils.data.distributed.DistributedSampler(
          valid_dataset,
          num_replicas=xm.xrt_world_size(),
          rank=xm.get_ordinal(),
          shuffle=False)

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=BATCH_SIZE,
        sampler=valid_sampler,
        drop_last=False,
        num_workers=0
    )
    device = xm.xla_device() 
    model = mx.to(device) 
    xm.master_print('done loading model')

    param_optimizer = list(model.named_parameters()) 
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

    xm.master_print('training on train dataset')
    
    lr = 0.5e-5 * xm.xrt_world_size() 
    num_train_steps = int(len(train_dataset) / BATCH_SIZE / xm.xrt_world_size() * EPOCHS) 
    
    optimizer = AdamW(optimizer_grouped_parameters, lr=lr) 
    
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=num_train_steps
    )
    xm.master_print(f'num_training_steps = {num_train_steps}, world_size={xm.xrt_world_size()}')
    
    
    for epoch in range(EPOCHS):
        gc.collect()
        para_loader = pl.ParallelLoader(train_data_loader, [device])
        xm.master_print('parallel loader created... training now')
        gc.collect()
        train_loop_fn(para_loader.per_device_loader(device), model, optimizer, device, scheduler=scheduler)
        del para_loader
        para_loader = pl.ParallelLoader(valid_data_loader, [device])
        gc.collect()
        o, t = eval_loop_fn(para_loader.per_device_loader(device), model, device)
        del para_loader
        gc.collect()
        auc = metrics.roc_auc_score(np.array(t) >= 0.5, o)
        auc_reduced = xm.mesh_reduce('auc_reduce',auc,reduce_fn)
        xm.master_print(f'AUC = {auc_reduced}')
        gc.collect()
    xm.save(model.state_dict(), "model.bin")

In [21]:
import time

def _mp_fn(rank, flags):
    a = _run()

FLAGS={}
start_time = time.time()
xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')

done loading model
training on train dataset
num_training_steps = 10035, world_size=8
parallel loader created... training now


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

step: 0 1.0, 0.4169921875
step: 50 0.8676470588235294, 0.33154296875
, accuracy = [0.8814489571899012]
AUC = 0.8376165790010087
parallel loader created... training now
step: 0 0.0, 2.6875
step: 50 0.6985294117647058, 0.389892578125
, accuracy = [0.7299670691547749]
AUC = 0.782102144737878
parallel loader created... training now
step: 0 0.0, 1.5908203125
step: 50 0.7365196078431373, 0.322265625
, accuracy = [0.7639956092206367]
AUC = 0.9994443072504372
parallel loader created... training now
step: 0 1.0, 0.30517578125
step: 50 0.9889705882352942, 0.2303466796875
, accuracy = [0.9901207464324918]
AUC = 0.9999871725220331
parallel loader created... training now
step: 0 1.0, 0.2076416015625
step: 50 0.9987745098039216, 0.2286376953125
, accuracy = [0.9978046103183315]
AUC = 0.999991511227816
